In [1]:
%load_ext autoreload
from pathlib import Path
import itertools

import torch

from src.data_loader_cifar import get_cifar_dataloaders
from src.measures_sharpness import calculate_sharpness_metrics
import src.model_constructor as constructor
import warnings
from copy import deepcopy

warnings.filterwarnings("ignore")

In [2]:
%autoreload 2
def models_iterator(depths, filters_sizes, optimizers, drops, lrs):
    models_to_train = []
    for depth in depths:
        fs = filters_sizes[str(depth)]
        d = drops[str(depth)]
        configurations = list(itertools.product(fs, optimizers, d, lrs))

        for config in configurations:
            filters_size, optimizer, drop, lr = config

            nr_filters = filters_size[:depth]
            conv_layers = constructor.Conv(nr_conv=depth, nr_filters=nr_filters, maxpool_batchnorm=True)
            fc_size = filters_size[depth:]
            act_fun = ["ReLU"] * depth
            dropouts = drop
            fc_layers = constructor.FC(
                nr_fc=depth,
                fc_size=fc_size,
                act_funs=act_fun,
                dropouts=dropouts,
                in_features=conv_layers.finaldim,
                num_classes=10,
                batchnorm=True,
            )

            # Create the model using the CNN constructor
            model = constructor.CNN(
                conv_layers=conv_layers, fc_layers=fc_layers, num_classes=10, lr=lr, optim=optimizer
            )

            # Store the model and its parameters in the list
            model_info = {"name": f"{model.name}", "model": model, "params": {"lr": lr, "optimizer": optimizer}}

            models_to_train.append(model_info)

    return models_to_train

def accuracy_score(model, train_dataloader, device):
    model.eval()
    correct = total = 0

    with torch.no_grad():
        for X, y in train_dataloader:
            X, y = X.to(device), y.to(device)
            preds = model(X).argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    return correct / total

depths = [2, 4]
filters_sizes = {
    "2": [[8, 16, 160, 80], [32, 64, 320, 160]],
    "4": [[4, 8, 16, 32, 200, 200, 160, 80], [8, 16, 32, 64, 400, 320, 160, 80]],
}
lrs = [0.01, 0.001, "scheduler"]
drops = {"2": [[0.0, 0.0], [0.5, 0.2]], "4": [[0.0] * 4, [0.5, 0.3, 0.3, 0.2]]}
optimizers = ["adam", "sgd"]


models_to_train = models_iterator(depths, filters_sizes, optimizers, drops, lrs)

In [3]:
train_dataloader, val_dataloader, test_dataloader = get_cifar_dataloaders(path=Path("data/processed/cifar"), batch_size=32)

In [ ]:
name = models_to_train[0]['name']
model = models_to_train[0]['model']

model_path = f"models/{name}"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {device}")

trained_model_0 = deepcopy(models_to_train[0]['model'])
untrained_model_0 = deepcopy(models_to_train[0]['model'])

trained_model_0.load_state_dict(torch.load(model_path+'/trained.pt', map_location=device))
untrained_model_0.load_state_dict(torch.load(model_path+'/untrained.pt', map_location=device))


trained_model_0.to(device)
untrained_model_0.to(device)

trained_model_0.eval()
accuracy_0 = accuracy_score(trained_model_0, train_dataloader, device)
metrics_0 = calculate_sharpness_metrics(trained_model_0, train_dataloader, accuracy_0)

Device: cuda
Accuracy: 0.96375
Search depth 0, sigma: 2.5
Monte Carlo iteration 0
Step: 0
Weights Norm before normalzing 611.6412420753312
Weights Norm after normalzing 2.5000000868691115
Deviation: 0.8627777777777778, Perturbed Accuracy: 0.10097222222222223
Step: 1
Weights Norm before normalzing 2.4999989097586317
Deviation: -0.0072685185185185075, Perturbed Accuracy: 0.9710185185185185
Step: 2
Weights Norm before normalzing 2.5000063598860884
Weights Norm after normalzing 2.500003637335877
Step: 3
Weights Norm before normalzing 2.500017977156069
Weights Norm after normalzing 2.500001478145277
Step: 4
Weights Norm before normalzing 2.50002191045208
Weights Norm after normalzing 2.50000099067673
Deviation: -0.006712962962962976, Perturbed Accuracy: 0.970462962962963
Step: 5
Weights Norm before normalzing 2.500028382627763
Weights Norm after normalzing 2.4999999464926073
Step: 6
Weights Norm before normalzing 2.5000406004596822
Weights Norm after normalzing 2.5000007813620555
Step: 7
We

In [ ]:
print(f"Accuracy: {accuracy_0}")
print(f"Metrics: {metrics_0}")

Accuracy: 0.96375
Metrics: {'Sharpness_Sigma': 2.5, 'Sharpness_Flatness': 0.16, 'Sharpness_Bound': 4.839417245035003}


In [ ]:
name = models_to_train[1]['name']
model = models_to_train[1]['model']

model_path = f"models/{name}"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

trained_model_1 = deepcopy(models_to_train[1]['model'])
untrained_model_1 = deepcopy(models_to_train[1]['model'])

trained_model_1.load_state_dict(torch.load(model_path+'/trained.pt', map_location=device))
untrained_model_1.load_state_dict(torch.load(model_path+'/untrained.pt', map_location=device))

train_dataloader, val_dataloader, test_dataloader = get_cifar_dataloaders(path=Path("data/processed/cifar"), batch_size=32)

trained_model_1.to(device)
untrained_model_1.to(device)

trained_model_1.eval()
accuracy_1 = accuracy_score(trained_model_1, train_dataloader, device)
metrics_1 = calculate_sharpness_metrics(trained_model_1, train_dataloader, accuracy_1)

Search depth 0, sigma: 2.5
Total L2 norm: 305.8198941013399
Early stopping triggered at ascent step 1 due to high deviation.
Perturbed accuracy: 0.1
Deviation: 0.79


Search depth 1, sigma: 1.25
Total L2 norm: 152.93942376159578
Early stopping triggered at ascent step 1 due to high deviation.
Perturbed accuracy: 0.1
Deviation: 0.79


Search depth 2, sigma: 0.625
Total L2 norm: 76.50673342187939
Early stopping triggered at ascent step 1 due to high deviation.
Perturbed accuracy: 0.1
Deviation: 0.79


Search depth 3, sigma: 0.3125
Total L2 norm: 38.221920456819106
Early stopping triggered at ascent step 1 due to high deviation.
Perturbed accuracy: 0.1750925925925926
Deviation: 0.7149074074074074


Search depth 4, sigma: 0.15625
Total L2 norm: 19.101761788882
Early stopping triggered at ascent step 1 due to high deviation.
Perturbed accuracy: 0.14907407407407408
Deviation: 0.740925925925926


Search depth 5, sigma: 0.078125
Total L2 norm: 9.566812153583099
Early stopping triggered at asce

In [20]:
print(f"Accuracy: {accuracy_1}")
print(f"Metrics: {metrics_1}")

Accuracy: 0.89
Metrics: {'Sharpness_MAG_Sigma': 0.015625, 'Sharpness_MAG_Flatness': 4096.0, 'Sharpness_MAG_Bound': 774.3067592056004}


In [ ]:
name = models_to_train[3]['name']
model = models_to_train[3]['model']

model_path = f"models/{name}"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

trained_model_3 = deepcopy(models_to_train[3]['model'])
untrained_model_3 = deepcopy(models_to_train[3]['model'])

trained_model_3.load_state_dict(torch.load(model_path+'/trained.pt', map_location=device))
untrained_model_3.load_state_dict(torch.load(model_path+'/untrained.pt', map_location=device))

train_dataloader, val_dataloader, test_dataloader = get_cifar_dataloaders(path=Path("data/processed/cifar"), batch_size=32)

trained_model_3.to(device)
untrained_model_3.to(device)

trained_model_3.eval()
accuracy_3 = accuracy_score(trained_model_3, train_dataloader, device)
metrics_3 = calculate_sharpness_metrics(trained_model_3, train_dataloader, accuracy_3)

Accuracy: 0.90375
Search depth 0, sigma: 2.5
Monte Carlo iteration 0
Step: 0
Weights Norm before normalzing 611.2427412058438
Weights Norm after normalzing 2.5000002288492293
Deviation: 0.7178240740740741, Perturbed Accuracy: 0.18592592592592594
Step: 1
Weights Norm before normalzing 2.500074739315126
Weights Norm after normalzing 2.500000508045144
Deviation: -4.6296296296266526e-05, Perturbed Accuracy: 0.9037962962962963
Step: 2
Weights Norm before normalzing 2.5001840692149786
Weights Norm after normalzing 2.4999999466526783
Step: 3
Weights Norm before normalzing 2.5003290328349985
Weights Norm after normalzing 2.4999999902705894
Step: 4
Weights Norm before normalzing 2.500558084903509
Weights Norm after normalzing 2.5000001852982607
Deviation: 0.0014351851851852615, Perturbed Accuracy: 0.9023148148148148
Step: 5
Weights Norm before normalzing 2.5008443147791612
Weights Norm after normalzing 2.499999934353399
Step: 6
Weights Norm before normalzing 2.5012340219786764
Weights Norm afte

In [ ]:
print(f"Accuracy: {accuracy_3}")
print(f"Metrics: {metrics_3}")

Accuracy: 0.90375
Metrics: {'Sharpness_Sigma': 2.5, 'Sharpness_Flatness': 0.16, 'Sharpness_Bound': 4.839417245035003}


In [15]:
name = models_to_train[6]['name']
model = models_to_train[6]['model']

model_path = f"models/{name}"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

trained_model_6 = deepcopy(models_to_train[6]['model'])
untrained_model_6 = deepcopy(models_to_train[6]['model'])

trained_model_6.load_state_dict(torch.load(model_path+'/trained.pt', map_location=device))
untrained_model_6.load_state_dict(torch.load(model_path+'/untrained.pt', map_location=device))

train_dataloader, val_dataloader, test_dataloader = get_cifar_dataloaders(path=Path("data/processed/cifar"), batch_size=62)

trained_model_6.to(device)
untrained_model_6.to(device)

trained_model_6.eval()
accuracy_6 = accuracy_score(trained_model_6, train_dataloader, device)
metrics_6 = calculate_sharpness_metrics(trained_model_6, train_dataloader, accuracy_6)

Accuracy: 0.7927314814814815
Search depth 0, sigma: 2.5
Monte Carlo iteration 0
Step: 0
Weights Norm before normalzing 611.5656972498446
Weights Norm after normalzing 2.499999897828091
Deviation: 0.6927314814814816, Perturbed Accuracy: 0.1
Step: 1
Weights Norm before normalzing 2.502014930356379
Weights Norm after normalzing 2.4999999901570846
Deviation: 0.05111111111111122, Perturbed Accuracy: 0.7416203703703703
Step: 2
Weights Norm before normalzing 2.5058608210684272
Weights Norm after normalzing 2.500000066094798
Step: 3
Weights Norm before normalzing 2.5153177571830283
Weights Norm after normalzing 2.499999998416752
Step: 4
Weights Norm before normalzing 2.5263097694190257
Weights Norm after normalzing 2.5000000049127267
Deviation: 0.5478240740740741, Perturbed Accuracy: 0.2449074074074074
Early stopping triggered.
Min accuracy: 0.2449074074074074, Deviation: 0.5478240740740741
Lower bound: 0.0, Upper bound: 2.5


Search depth 1, sigma: 1.25
Monte Carlo iteration 0
Step: 0
Weights

In [ ]:
print(f"Accuracy: {accuracy_6}")
print(f"Metrics: {metrics_6}")

Accuracy: 0.7927314814814815
Metrics: {'Sharpness_Sigma': 0.009765625, 'Sharpness_Flatness': 10485.76, 'Sharpness_Bound': 1238.8908147289608}


In [ ]:
name = models_to_train[8]['name']
model = models_to_train[8]['model']

model_path = f"models/{name}"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

trained_model_8 = deepcopy(models_to_train[8]['model'])
untrained_model_8 = deepcopy(models_to_train[8]['model'])

trained_model_8.load_state_dict(torch.load(model_path+'/trained.pt', map_location=device))
untrained_model_8.load_state_dict(torch.load(model_path+'/untrained.pt', map_location=device))

train_dataloader, val_dataloader, test_dataloader = get_cifar_dataloaders(path=Path("data/processed/cifar"), batch_size=82)

trained_model_8.to(device)
untrained_model_8.to(device)

trained_model_8.eval()
accuracy_8 = accuracy_score(trained_model_8, train_dataloader, device)
metrics_8 = calculate_sharpness_metrics(trained_model_8, train_dataloader, accuracy_8)

Accuracy: 0.9179166666666667
Search depth 0, sigma: 2.5
Monte Carlo iteration 0
Step: 0
Weights Norm before normalzing 611.2107325393315
Weights Norm after normalzing 2.499999797949559
Deviation: 0.8179166666666667, Perturbed Accuracy: 0.1
Step: 1
Weights Norm before normalzing 2.501332873604473
Weights Norm after normalzing 2.5000000125553923
Deviation: 0.006898148148148153, Perturbed Accuracy: 0.9110185185185186
Step: 2
Weights Norm before normalzing 2.5026079424099628
Weights Norm after normalzing 2.499999983754242
Step: 3
Weights Norm before normalzing 2.5182218334169
Weights Norm after normalzing 2.4999998971485162
Step: 4
Weights Norm before normalzing 2.53130932274195
Weights Norm after normalzing 2.5000000504660416
Deviation: 0.6983796296296296, Perturbed Accuracy: 0.21953703703703703
Early stopping triggered.
Min accuracy: 0.21953703703703703, Deviation: 0.6983796296296296
Lower bound: 0.0, Upper bound: 2.5


Search depth 1, sigma: 1.25
Monte Carlo iteration 0
Step: 0
Weights 

KeyboardInterrupt: 

In [14]:
print(f"Accuracy: {accuracy_8}")
print(f"Metrics: {metrics_8}")

Accuracy: 0.9179166666666667
Metrics: {'Sharpness_Sigma': 0.01953125, 'Sharpness_Flatness': 2621.44, 'Sharpness_Bound': 619.4454073644804}
